In [2]:
# 6. Triển khai Support Vector Machine (SVM)

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC # Import lớp SVM cho phân loại
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import sys
import os
import joblib # Dùng để lưu/tải mô hình

# --- Đảm bảo có thể import từ thư mục src ---
# Thêm thư mục gốc của dự án vào đường dẫn Python
module_path = os.path.abspath(os.path.join('..')) # Giả sử đang chạy từ notebooks hoặc tương tự
if module_path not in sys.path:
    sys.path.append(module_path)

# --- Import hàm tiền xử lý từ src ---
try:
    # Giả sử hàm tiền xử lý của bạn tên là clean_text_v1 trong file preprocess.py
    from src.preprocess import clean_text_v1
except ImportError:
    print("Lỗi: Không thể import hàm clean_text_v1 từ src/preprocess.py.")
    print("Hãy đảm bảo file tồn tại và chứa hàm cần thiết.")
    exit()
except Exception as e:
    print(f"Lỗi khác khi import hàm tiền xử lý: {e}")
    exit()

# --- 1. Đọc và Chuẩn bị Dữ liệu ---
print("Đang đọc dữ liệu...")
try:
    # Sử dụng đường dẫn tương đối nếu cần
    data_path = '../data/processed/spam_cleaned_columns.csv'
    df = pd.read_csv(data_path, encoding='latin-1')
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file {data_path}")
    exit()



# --- 2. Áp dụng Tiền xử lý ---
print("Đang áp dụng tiền xử lý văn bản...")
df['Cleaned_Message'] = df['Message'].apply(clean_text_v1)
print("Tiền xử lý văn bản hoàn tất.")
print("5 dòng đầu với Cleaned_Message:")
print(df[['Message', 'Cleaned_Message', 'Category']].head())

# --- 3. Chuẩn bị Dữ liệu cho Mô hình ---
X = df['Cleaned_Message']
y = df['Category_Num']



Đang đọc dữ liệu...
Đang áp dụng tiền xử lý văn bản...
Tiền xử lý văn bản hoàn tất.
5 dòng đầu với Cleaned_Message:
                                             Message  \
0  Go until jurong point, crazy.. Available only ...   
1                      Ok lar... Joking wif u oni...   
2  Free entry in 2 a wkly comp to win FA Cup fina...   
3  U dun say so early hor... U c already then say...   
4  Nah I don't think he goes to usf, he lives aro...   

                                     Cleaned_Message Category  
0  go jurong point crazy available bugis n great ...      ham  
1                            ok lar joking wif u oni      ham  
2  free entry 2 wkly comp win fa cup final tkts 2...     spam  
3                u dun say early hor u c already say      ham  
4        nah dont think goes usf lives around though      ham  


In [14]:
# --- 4. Chia Dữ liệu Train/Test ---
# Sử dụng cùng random_state và stratify như khi làm Naive Bayes để đảm bảo tính nhất quán
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
#stratify: đảm bảo tỷ lệ của từng lớp trong tập huấn luyện và kiểm tra giống nhau
print(f"\nĐã chia dữ liệu: {X_train.shape[0]} huấn luyện, {X_test.shape[0]} kiểm tra")

# --- 5. Vector hóa TF-IDF ---
# Sử dụng lại cài đặt TF-IDF giống như với Naive Bayes
print("Đang thực hiện vector hóa TF-IDF...")
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train) 

#fit_transform: tạo ra vector cho tập huấn luyện


print("Đang lấy danh sách tên đặc trưng...")
try:
    feature_names = tfidf_vectorizer.get_feature_names_out()
    print(f"Số lượng đặc trưng: {len(feature_names)}")
    # print("Một vài tên đặc trưng đầu tiên:", feature_names[:20]) # In ra 20 đặc trưng đầu tiên để kiểm tra
    
    # KIỂM TRA CÁC N-GRAMS QUAN TRỌNG TỪ HAI CÂU SPAM
    print("\nKiểm tra các n-grams quan trọng:")
    important_ngrams = ["limited time", "time offer", "exclusive discount", 
                          "50 percent", "percent discount", "this friday", "friday only",
                          "limited", "offer", "exclusive", "discount", "percent", "friday", "only"] # Thêm cả unigrams
    
    found_ngrams = []
    for ngram in important_ngrams:
        if ngram in feature_names:
            found_ngrams.append(ngram)
            print(f"'{ngram}' CÓ trong từ điển.")
        else:
            print(f"'{ngram}' KHÔNG CÓ trong từ điển.")
    
    if not found_ngrams:
        print("Không tìm thấy n-gram quan trọng nào trong từ điển với max_features=5000 và ngram_range=(1,2).")
        print("Hãy thử tăng max_features hoặc sử dụng min_df thay thế.")

except Exception as e:
    print(f"Lỗi khi lấy tên đặc trưng: {e}")


X_test_tfidf = tfidf_vectorizer.transform(X_test)
#transform: tạo ra vector cho tập kiểm tra
#fit_transform gồm 2 bước fit và transform
#fit: tạo ra vectorizer (bộ từ vựng từ tất cả các từ trong tập huấn luyện)
#transform: tạo ra vector cho tập huấn luyện và kiểm tra
#Không sử dụng fit_transform để tránh rò rỉ dữ liệu test cho mô hình, chỉ sử dụng fit để tạo ra vectorizer


print(f"Kích thước ma trận TF-IDF tập huấn luyện: {X_train_tfidf.shape}")
print(f"Kích thước ma trận TF-IDF tập kiểm tra: {X_test_tfidf.shape}")

# --- 6. Huấn luyện Mô hình SVM ---
print("\nBắt đầu huấn luyện mô hình SVM...")
# Khởi tạo mô hình SVM (SVC - Support Vector Classifier)
# kernel='linear': Thường hiệu quả cho dữ liệu văn bản.
# probability=True: Cho phép sử dụng predict_proba sau này (có thể làm chậm huấn luyện một chút).
# random_state=42: Để đảm bảo kết quả có thể lặp lại (nếu thuật toán có yếu tố ngẫu nhiên).
svm_model = SVC(kernel='linear', probability=True, random_state=42, class_weight='balanced')

# Huấn luyện mô hình
svm_model.fit(X_train_tfidf, y_train)
print("Đã huấn luyện xong mô hình SVM.")

#    Giả sử có 3 từ trong từ điển: "free", "prize", "meeting"
   
#    Spam messages:
#    - "Get free prize now!"     -> [0.8, 0.7, 0.0]
#    - "Win free prize today!"   -> [0.7, 0.8, 0.0]
#    - "Claim your free prize!"  -> [0.9, 0.6, 0.0]
   
#    Ham messages:
#    - "Meeting at 2pm"          -> [0.0, 0.0, 0.9]
#    - "Schedule meeting"        -> [0.0, 0.0, 0.8]
#    - "Cancel meeting"          -> [0.0, 0.0, 0.7]

#Các vector spam tập trung ở 1 vùng (có nhiều từ "free", "prize")
#Các vector ham tập trung ở 1 vùng (có nhiều từ "meeting")
#Dễ dàng tìm được siêu phẳng phân tíchtích

# --- 7. Dự đoán trên Tập Kiểm tra ---
y_pred_svm = svm_model.predict(X_test_tfidf)

# --- 8. Đánh giá Mô hình ---
print("\nKết quả đánh giá SVM (kernel='linear', C=mặc định):")
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_svm))
# target_names giúp hiển thị tên lớp 'ham', 'spam' thay vì 0, 1
print("\nClassification Report:\n", classification_report(y_test, y_pred_svm, target_names=['ham', 'spam']))

# --- (Tùy chọn) Lưu mô hình và vectorizer ---
# print("\nĐang lưu mô hình và vectorizer...")
# output_dir = '../results/trained_models/'
# os.makedirs(output_dir, exist_ok=True) # Tạo thư mục nếu chưa có
# joblib.dump(svm_model, os.path.join(output_dir, 'svm_linear_model.pkl'))
# joblib.dump(tfidf_vectorizer, os.path.join(output_dir, 'tfidf_vectorizer.pkl'))
# print("Đã lưu mô hình và vectorizer vào thư mục results/trained_models/")

# --- (Tùy chọn) Lấy xác suất dự đoán ---
# y_pred_proba_svm = svm_model.predict_proba(X_test_tfidf)
# print("\nXác suất dự đoán cho 5 mẫu đầu tiên:\n", y_pred_proba_svm[:5])


Đã chia dữ liệu: 4457 huấn luyện, 1115 kiểm tra
Đang thực hiện vector hóa TF-IDF...
Đang lấy danh sách tên đặc trưng...
Số lượng đặc trưng: 5000

Kiểm tra các n-grams quan trọng:
'limited time' KHÔNG CÓ trong từ điển.
'time offer' KHÔNG CÓ trong từ điển.
'exclusive discount' KHÔNG CÓ trong từ điển.
'50 percent' KHÔNG CÓ trong từ điển.
'percent discount' KHÔNG CÓ trong từ điển.
'this friday' KHÔNG CÓ trong từ điển.
'friday only' KHÔNG CÓ trong từ điển.
'limited' KHÔNG CÓ trong từ điển.
'offer' CÓ trong từ điển.
'exclusive' KHÔNG CÓ trong từ điển.
'discount' CÓ trong từ điển.
'percent' KHÔNG CÓ trong từ điển.
'friday' CÓ trong từ điển.
'only' KHÔNG CÓ trong từ điển.
Kích thước ma trận TF-IDF tập huấn luyện: (4457, 5000)
Kích thước ma trận TF-IDF tập kiểm tra: (1115, 5000)

Bắt đầu huấn luyện mô hình SVM...
Đã huấn luyện xong mô hình SVM.

Kết quả đánh giá SVM (kernel='linear', C=mặc định):
Accuracy: 0.9847533632286996

Confusion Matrix:
 [[960   6]
 [ 11 138]]

Classification Report:
  

In [15]:
# --- (Tùy chọn) Lưu mô hình và vectorizer ---
print("\nĐang lưu mô hình và vectorizer...")
output_dir = '../results/trained_models/'
os.makedirs(output_dir, exist_ok=True) # Tạo thư mục nếu chưa có
joblib.dump(tfidf_vectorizer, os.path.join(output_dir, 'tfidf_vectorizer.pkl'))
print("Đã lưu mô hình và vectorizer vào thư mục results/trained_models/")


Đang lưu mô hình và vectorizer...
Đã lưu mô hình và vectorizer vào thư mục results/trained_models/


In [19]:
output_dir = '../results/trained_models/'
os.makedirs(output_dir, exist_ok=True)
joblib.dump(svm_model, os.path.join(output_dir, 'svm_model.pkl'))
print("Đã lưu mô hình SVM tốt nhất.")


Đã lưu mô hình SVM tốt nhất.


In [16]:
# -*- coding: utf-8 -*-
# Cell mới: Tạo Test Case và Dự đoán bằng mô hình SVM đã huấn luyện

# --- Giả định các biến sau đã tồn tại từ cell trước ---
# svm_model: Đối tượng mô hình SVC đã được .fit()
# tfidf_vectorizer: Đối tượng TfidfVectorizer đã được .fit_transform() trên tập train
# clean_text_v1: Hàm tiền xử lý văn bản đã được định nghĩa hoặc import

# --- Định nghĩa các tin nhắn mới cần dự đoán ---
new_messages_for_test = [
    "URGENT! Claim your FREE £1000 prize now! Click http://spamlink.com", # Tin nhắn có vẻ là SPAM
    "Hey, wondering if you're free for coffee later today?", # Tin nhắn có vẻ là HAM
    "Meet singles in your area, text DATE to 88888 T&Cs apply 18+", # Tin nhắn có vẻ là SPAM
    "Remember to buy bread and eggs on your way back.", # Tin nhắn có vẻ là HAM
    "Limited time offer! Exclusive discount just for you!", # Tin nhắn có vẻ là SPAM
    "50 percent discount this Friday only!" #SPAM
]

print("--- Bắt đầu Test Case ---")
print(f"Số lượng tin nhắn thử nghiệm: {len(new_messages_for_test)}")

# --- 1. Tiền xử lý các tin nhắn mới ---
# Áp dụng hàm tiền xử lý đã có (cần đảm bảo hàm clean_text_v1 đã được định nghĩa/import ở cell trước)
try:
    cleaned_test_messages = [clean_text_v1(msg) for msg in new_messages_for_test]
    print("\nTin nhắn sau khi tiền xử lý:")
    for i, msg in enumerate(cleaned_test_messages):
        print(f"{i+1}: {msg}")
except NameError:
    print("\nLỗi: Hàm 'clean_text_v1' chưa được định nghĩa hoặc import.")
    print("Vui lòng chạy cell định nghĩa/import hàm này trước.")
    # Dừng thực thi cell này nếu hàm chưa có
    raise

# --- 2. Vector hóa các tin nhắn mới ---
# *** Quan trọng: Dùng đối tượng tfidf_vectorizer đã fit từ cell trước, chỉ gọi .transform() ***
try:
    new_messages_tfidf_test = tfidf_vectorizer.transform(cleaned_test_messages)
    print(f"\nĐã vector hóa {len(cleaned_test_messages)} tin nhắn thành ma trận TF-IDF kích thước: {new_messages_tfidf_test.shape}")
except NameError:
    print("\nLỗi: Biến 'tfidf_vectorizer' chưa được định nghĩa.")
    print("Vui lòng chạy cell huấn luyện TF-IDF Vectorizer trước.")
    raise

# --- 3. Dự đoán ---
print("\nĐang thực hiện dự đoán bằng mô hình SVM...")
try:
    # Dự đoán nhãn (0 hoặc 1)
    predictions_test = svm_model.predict(new_messages_tfidf_test)
    # Dự đoán xác suất (P(ham), P(spam))
    # Lưu ý: svm_model phải được khởi tạo với probability=True ở cell huấn luyện
    probabilities_test = svm_model.predict_proba(new_messages_tfidf_test)
except NameError:
    print("\nLỗi: Biến 'svm_model' chưa được định nghĩa.")
    print("Vui lòng chạy cell huấn luyện mô hình SVM trước.")
    raise
except AttributeError:
     print("\nLỗi: Không thể lấy xác suất.")
     print("Để dùng predict_proba(), mô hình SVM cần được khởi tạo với tham số 'probability=True'.")
     # Gán giá trị mặc định để code không bị lỗi tiếp
     probabilities_test = np.array([[0.5, 0.5]] * len(predictions_test)) # Tạo mảng xác suất giả định

# --- 4. Hiển thị kết quả ---
print("\n--- Kết quả Dự đoán Test Case ---")
label_map = {0: 'ham', 1: 'spam'} # Ánh xạ nhãn số về chữ

for i, original_message in enumerate(new_messages_for_test):
    predicted_label_num = predictions_test[i]
    predicted_label_text = label_map[predicted_label_num]
    prob_ham = probabilities_test[i][0] # Xác suất là ham (lớp 0)
    prob_spam = probabilities_test[i][1] # Xác suất là spam (lớp 1)

    print(f"\nTin nhắn gốc : \"{original_message}\"")
    # print(f"Tin nhắn sạch: \"{cleaned_test_messages[i]}\"") # Bỏ comment nếu muốn xem
    print(f"  -> Dự đoán   : {predicted_label_text.upper()}")
    # Chỉ in xác suất nếu lấy được
    if 'AttributeError' not in locals() or not isinstance(locals()['AttributeError'], AttributeError):
         print(f"  -> Xác suất : [Ham={prob_ham:.4f}, Spam={prob_spam:.4f}]")

print("\n--- Hoàn thành Test Case ---")

--- Bắt đầu Test Case ---
Số lượng tin nhắn thử nghiệm: 6

Tin nhắn sau khi tiền xử lý:
1: urgent claim free 1000 prize click httpspamlinkcom
2: hey wondering youre free coffee later today
3: meet singles area text date 88888 tcs apply 18
4: remember buy bread eggs way back
5: limited time offer exclusive discount
6: 50 percent discount friday

Đã vector hóa 6 tin nhắn thành ma trận TF-IDF kích thước: (6, 5000)

Đang thực hiện dự đoán bằng mô hình SVM...

--- Kết quả Dự đoán Test Case ---

Tin nhắn gốc : "URGENT! Claim your FREE £1000 prize now! Click http://spamlink.com"
  -> Dự đoán   : SPAM
  -> Xác suất : [Ham=0.0063, Spam=0.9937]

Tin nhắn gốc : "Hey, wondering if you're free for coffee later today?"
  -> Dự đoán   : HAM
  -> Xác suất : [Ham=0.9988, Spam=0.0012]

Tin nhắn gốc : "Meet singles in your area, text DATE to 88888 T&Cs apply 18+"
  -> Dự đoán   : SPAM
  -> Xác suất : [Ham=0.0204, Spam=0.9796]

Tin nhắn gốc : "Remember to buy bread and eggs on your way back."
  -> Dự đoán

B1:Tạo 1 file google doc -> Note lại toàn bộ kết quả baseline của từng thuật toán (accuracy, precision, recall, f1-score, confusion matrix)
B2: Tinh chỉnh siêu tham số
- random forest
- Logistic Regression
B3: Sử dụng GridSearchCV hoặc RandomizedSearchCV (CV: Cross validation)
- Cross Validation: K-Fold Cross Validation (Kiểm định chéo K lần)
  + Chia dữ liệu: Chia ngẫu nhiên tập train thành K phần, giá trị K thường là 5 hoặc 10
  + Lặp K lần: Quá trình huấn luyện và đánh giá dược lặp lại K lần. Trong mỗi lần lặp (ví dụ lần lặp thứ i):
    > Chọn Fold kiểm tra: Fold thứ i được giữ lại làm tập dữ liệu kiểm định (validation set) hoặc là "test fold" cho lần lặp này.
    >Chọn fold huấn luyện: K-1 phần còn lại được gộp lại để làm tập dữ liệu huấn luyện.
    >Huấn luyện: Mô hình học máy được huấn luyện trên tập huấn luyện (k-1 folds)
    > Đánh giá: Mô hình vừa huấn luyện xong được đánh giá hiệu năng (tính f1-score, accuracy,..) trên tập kiểm định (fold thứ i). Kết quả đánh giá được lưu lại.
  + Tổng hợp kết quả: Sau khi thực hiện K lần lặp thì có K kết quả đánh giá. Hiệu năng cuối cùng của mô hình theo phương pháp CV thường được tính bằng lấy trung bình của K kết quả.
- GridSearchCV: thử tất cả các tổ hợp
- RandomizedSearchCV: thử một số tổ hợp tham số ngẫu nhiên.

B4:Cập nhật kết quả: ghi lại chỉ số hiệu năng của mô hình đã tune vào bảng so sánh chung (file google doc ở trên).


In [20]:
from sklearn.model_selection import GridSearchCV


print("Bắt đầu tinh chỉnh siêu tham số cho SVM")

# 1. Define parameters grid
param_grid_svm = {
  'C': [0.1, 1, 10, 100], # Regularization parameter
  'kernel': ['linear', 'rbf'], #linear kernel and rbf
  'gamma': ['scale', 'auto', 0.1, 1] #parameters for kernel RBF (skipped if kernel is linear)
  
}

# 2. Init GridSearchCV
grid_search_svm = GridSearchCV(estimator=SVC(probability=True, random_state=42, class_weight='balanced'),
                               param_grid=param_grid_svm,
                               cv = 5,
                               scoring= 'recall',
                            
                               verbose=2,
                               )

# 3. Fit GridSearchCV into training data
print("Running GridSearchCV on SVM...")
grid_search_svm.fit(X_train_tfidf, y_train)

# 4. Take out the best parameters and scores
print("\nTham số tốt nhất tìm được cho SVM:")
print(grid_search_svm.best_params_)
print(f"Điểm F1 (cross-validation) tốt nhất cho SVM: {grid_search_svm.best_score_:.4f}")

# 5. Lấy mô hình SVM tốt nhất
best_svm_model = grid_search_svm.best_estimator_

# 6. Đánh giá mô hình SVM tốt nhất trên tập Test
print("\nĐánh giá mô hình SVM tốt nhất trên tập Test:")
y_pred_svm_tuned = best_svm_model.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred_svm_tuned))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_svm_tuned))
print("\nClassification Report (SVM Tuned):\n", classification_report(y_test, y_pred_svm_tuned, target_names=['ham', 'spam']))

# (Tùy chọn) Lưu mô hình SVM tốt nhất



Bắt đầu tinh chỉnh siêu tham số cho SVM
Running GridSearchCV on SVM...
Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   2.3s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   2.2s


KeyboardInterrupt: 

In [18]:
output_dir = '../results/trained_models/'
os.makedirs(output_dir, exist_ok=True)
joblib.dump(best_svm_model, os.path.join(output_dir, 'best_svm_model.pkl'))
print("Đã lưu mô hình SVM tốt nhất.")

Đã lưu mô hình SVM tốt nhất.


In [7]:

new_messages_for_test = [
    "URGENT! Claim your FREE £1000 prize now! Click http://spamlink.com", # Tin nhắn có vẻ là SPAM
    "Hey, wondering if you're free for coffee later today?", # Tin nhắn có vẻ là HAM
    "Meet singles in your area, text DATE to 88888 T&Cs apply 18+", # Tin nhắn có vẻ là SPAM
    "Remember to buy bread and eggs on your way back.", # Tin nhắn có vẻ là HAM
    "Limited time offer! Exclusive discount just for you!", # Tin nhắn có vẻ là SPAM
    "50 percent discount this Friday only!"
]

print("--- Bắt đầu Test Case ---")
print(f"Số lượng tin nhắn thử nghiệm: {len(new_messages_for_test)}")

# --- 1. Tiền xử lý các tin nhắn mới ---
# Áp dụng hàm tiền xử lý đã có (cần đảm bảo hàm clean_text_v1 đã được định nghĩa/import ở cell trước)
try:
    cleaned_test_messages = [clean_text_v1(msg) for msg in new_messages_for_test]
    print("\nTin nhắn sau khi tiền xử lý:")
    for i, msg in enumerate(cleaned_test_messages):
        print(f"{i+1}: {msg}")
except NameError:
    print("\nLỗi: Hàm 'clean_text_v1' chưa được định nghĩa hoặc import.")
    print("Vui lòng chạy cell định nghĩa/import hàm này trước.")
    # Dừng thực thi cell này nếu hàm chưa có
    raise

# --- 2. Vector hóa các tin nhắn mới ---
# *** Quan trọng: Dùng đối tượng tfidf_vectorizer đã fit từ cell trước, chỉ gọi .transform() ***
try:
    new_messages_tfidf_test = tfidf_vectorizer.transform(cleaned_test_messages)
    print(f"\nĐã vector hóa {len(cleaned_test_messages)} tin nhắn thành ma trận TF-IDF kích thước: {new_messages_tfidf_test.shape}")
except NameError:
    print("\nLỗi: Biến 'tfidf_vectorizer' chưa được định nghĩa.")
    print("Vui lòng chạy cell huấn luyện TF-IDF Vectorizer trước.")
    raise

# --- 3. Dự đoán ---
print("\nĐang thực hiện dự đoán bằng mô hình SVM...")
try:
    # Dự đoán nhãn (0 hoặc 1)
    predictions_test = svm_model.predict(new_messages_tfidf_test)
    # Dự đoán xác suất (P(ham), P(spam))
    # Lưu ý: svm_model phải được khởi tạo với probability=True ở cell huấn luyện
    probabilities_test = best_svm_model.predict_proba(new_messages_tfidf_test)
except NameError:
    print("\nLỗi: Biến 'svm_model' chưa được định nghĩa.")
    print("Vui lòng chạy cell huấn luyện mô hình SVM trước.")
    raise
except AttributeError:
     print("\nLỗi: Không thể lấy xác suất.")
     print("Để dùng predict_proba(), mô hình SVM cần được khởi tạo với tham số 'probability=True'.")
     # Gán giá trị mặc định để code không bị lỗi tiếp
     probabilities_test = np.array([[0.5, 0.5]] * len(predictions_test)) # Tạo mảng xác suất giả định

# --- 4. Hiển thị kết quả ---
print("\n--- Kết quả Dự đoán Test Case ---")
label_map = {0: 'ham', 1: 'spam'} # Ánh xạ nhãn số về chữ

for i, original_message in enumerate(new_messages_for_test):
    predicted_label_num = predictions_test[i]
    predicted_label_text = label_map[predicted_label_num]
    prob_ham = probabilities_test[i][0] # Xác suất là ham (lớp 0)
    prob_spam = probabilities_test[i][1] # Xác suất là spam (lớp 1)

    print(f"\nTin nhắn gốc : \"{original_message}\"")
    # print(f"Tin nhắn sạch: \"{cleaned_test_messages[i]}\"") # Bỏ comment nếu muốn xem
    print(f"  -> Dự đoán   : {predicted_label_text.upper()}")
    # Chỉ in xác suất nếu lấy được
    if 'AttributeError' not in locals() or not isinstance(locals()['AttributeError'], AttributeError):
         print(f"  -> Xác suất : [Ham={prob_ham:.4f}, Spam={prob_spam:.4f}]")

print("\n--- Hoàn thành Test Case ---")

--- Bắt đầu Test Case ---
Số lượng tin nhắn thử nghiệm: 6

Tin nhắn sau khi tiền xử lý:
1: urgent claim free prize click httpspamlinkcom
2: hey wondering youre free coffee later today
3: meet singles area text date tcs apply
4: remember buy bread eggs way back
5: limited time offer exclusive discount
6: percent discount friday

Đã vector hóa 6 tin nhắn thành ma trận TF-IDF kích thước: (6, 5000)

Đang thực hiện dự đoán bằng mô hình SVM...

--- Kết quả Dự đoán Test Case ---

Tin nhắn gốc : "URGENT! Claim your FREE £1000 prize now! Click http://spamlink.com"
  -> Dự đoán   : SPAM
  -> Xác suất : [Ham=0.0000, Spam=1.0000]

Tin nhắn gốc : "Hey, wondering if you're free for coffee later today?"
  -> Dự đoán   : HAM
  -> Xác suất : [Ham=0.9993, Spam=0.0007]

Tin nhắn gốc : "Meet singles in your area, text DATE to 88888 T&Cs apply 18+"
  -> Dự đoán   : SPAM
  -> Xác suất : [Ham=0.0076, Spam=0.9924]

Tin nhắn gốc : "Remember to buy bread and eggs on your way back."
  -> Dự đoán   : HAM
  -> Xác

In [15]:
vec = tfidf_vectorizer.transform(["percent discount friday"])
print(vec)
# Hoặc kiểm tra giá trị cụ thể của từng từ
feature_names = tfidf_vectorizer.get_feature_names_out()
for word in ["percent", "discount", "friday"]:
    if word in tfidf_vectorizer.vocabulary_:
        idx = tfidf_vectorizer.vocabulary_[word]
        print(f"{word}: {vec[0, idx]}")

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3 stored elements and shape (1, 5000)>
  Coords	Values
  (0, 1108)	0.5298416056073167
  (0, 1527)	0.5237108275551885
  (0, 3536)	0.6670793371623199
percent: 0.6670793371623199
discount: 0.5298416056073167
friday: 0.5237108275551885


Trong NLP, có rất nhiều cách để vector hoá 1 văn bản text thành ma trận dạng số: Bag_of_words, TF-IDF, word embeddings,..
- Bag_of_words: "I arrive home at 6 o'clock" => [1,1,1,1]
"I come back home and then I go to my grandpa's house" 
- TF-IDF: Term Frequency-Inverse Document Frequency
Term Frequency (Bag_of_words) * Inverse Document Frequency 
IDF = log(number of documents / number of documents in which this word exists)

VD: 'Tôi đi học'
'Tôi đi chơi'
'Mua đồ ăn'

IDF = log(3 / 2) = 0.58 

